## import and clean

> Note - the solution here might be to pickle the original file and not lose all the file formats via csv. But the pickle runs into recurion problems at the moment.

> Note - this df is not finished (people and affil still have to be extracted from scraped data) but I can make a start

In [75]:
import pandas as pd
from ast import literal_eval

In [3]:
df = pd.read_csv('iot_df_halfclean.csv')
print(df.shape)
df.sample(5)

(876, 7)


,subject,summary,date_str,featured_in_tags,address,date_dt,day
615,Magnetism,Melvyn Bragg and guests discuss the history of...,29-09-2005,['Science'],https://www.bbc.co.uk/programmes/p003k9dd,2005-09-29,Thursday
356,Judas Maccabeus,Melvyn Bragg and his guests discuss the revolu...,24-11-2011,"['Religion', 'In Our Time: History - Top 10']",https://www.bbc.co.uk/programmes/b017cjm8,2011-11-24,Thursday
782,Science and Religion,Melvyn Bragg and guests discuss the areas of c...,25-01-2001,['Religion'],https://www.bbc.co.uk/programmes/p005479y,2001-01-25,Thursday
405,Sturm und Drang,Melvyn Bragg and his guests discuss the artist...,14-10-2010,"['Enlightenment', '18th Century', 'Culture']",https://www.bbc.co.uk/programmes/b00v72x6,2010-10-14,Thursday
49,Nero,Melvyn Bragg and guests discuss the life of Ne...,25-04-2019,"['Ancient Rome', 'History']",https://www.bbc.co.uk/programmes/m0004cp7,2019-04-25,Thursday


In [7]:
# fix date time

df['date_dt'] = df['date_dt'].astype('datetime64[ns]')

In [71]:
# function to covert string to lists in tags col

def sl (x):
    x = literal_eval(x)
    return x

# use function to make new col

df['tags'] = df.featured_in_tags.apply(sl)

In [77]:
# drop old tags col

df.drop(df.columns[3], axis=1, inplace = True)

## initial querying

In [4]:
# how many topics?

df.subject.nunique()

# One subject - artificial intel - is run twice

875

In [18]:
# how many years have we got?

df.date_dt.dt.year.nunique()

23

In [22]:
# how many progs each year? Around 40

df.groupby(df.date_dt.dt.year)['subject'].count()

date_dt
1998    12
1999    44
2000    34
2001    33
2002    38
2003    36
2004    40
2005    37
2006    42
2007    42
2008    41
2009    42
2010    41
2011    43
2012    43
2013    40
2014    38
2015    41
2016    39
2017    34
2018    37
2019    39
2020    40
Name: subject, dtype: int64

In [27]:
# filter for 2020

df[df.date_dt.dt.year == 2020].sort_values('date_dt').head()

,subject,summary,date_str,featured_in_tags,address,date_dt,day
23,Catullus,Melvyn Bragg and guests discuss Catullus (c84-...,09-01-2020,"['Ancient Rome', 'Culture']",https://www.bbc.co.uk/programmes/m000d1rg,2020-01-09,Thursday
22,The Siege of Paris 1870-71,Melvyn Bragg and guests discuss the siege of P...,16-01-2020,"['19th Century', 'History']",https://www.bbc.co.uk/programmes/m000d8rv,2020-01-16,Thursday
21,Solar Wind,Melvyn Bragg and guests discuss the flow of pa...,23-01-2020,['Science'],https://www.bbc.co.uk/programmes/m000dg9n,2020-01-23,Thursday
20,Alcuin,Melvyn Bragg and guests discuss Alcuin of York...,30-01-2020,"['Early Middle Ages', 'History']",https://www.bbc.co.uk/programmes/m000dqy8,2020-01-30,Thursday
19,George Sand,Melvyn Bragg and guests discuss the works and ...,06-02-2020,"['19th Century', 'Culture']",https://www.bbc.co.uk/programmes/m000dxsr,2020-02-06,Thursday


## subject tags

In [152]:
# how many different tags are used?

tag_list = []

for x in range(len(df)):
    z = df.iat[x, 6]
    for y in z:
        if y not in tag_list:
            tag_list.append(y)

len(tag_list)

146

In [165]:
# show the rows where tag lists are empty

i_list = []

for x in range(len(df)):
    z = df.iat[x, 6]
    if len(z) == 0:
        i_list.append(x)

df[df.index.isin(i_list)]

,subject,summary,date_str,address,date_dt,day,tags
295,Japan's Sakoku Period,Melvyn Bragg and his guests discuss Japan's Sa...,04-04-2013,https://www.bbc.co.uk/programmes/b01rlptf,2013-04-04,Thursday,[]
331,Marco Polo,Melvyn Bragg and his guests discuss the celebr...,24-05-2012,https://www.bbc.co.uk/programmes/b01hxpxh,2012-05-24,Thursday,[]
332,Clausewitz and On War,"Melvyn Bragg and his guests discuss On War, a ...",17-05-2012,https://www.bbc.co.uk/programmes/b01hl293,2012-05-17,Thursday,[]
335,The Battle of Bosworth Field,Melvyn Bragg and his guests discuss the Battle...,26-04-2012,https://www.bbc.co.uk/programmes/b01ghc43,2012-04-26,Thursday,[]
357,Ptolemy and Ancient Astronomy,Melvyn Bragg and his guests discuss the ancien...,17-11-2011,https://www.bbc.co.uk/programmes/b017528d,2011-11-17,Thursday,[]
360,The Siege of Tenochtitlan,Melvyn Bragg and his guests discuss the Siege ...,27-10-2011,https://www.bbc.co.uk/programmes/b016924x,2011-10-27,Thursday,[]
364,The Etruscan Civilisation,Melvyn Bragg and his guests discuss the Etrusc...,29-09-2011,https://www.bbc.co.uk/programmes/b0151q7j,2011-09-29,Thursday,[]


In [195]:
# filter by tag

def filt_list(d, tag):
    """takes 2 args - a dataframe (with a tags column!) and a tag - and returns rows in the df which contain the tag in the taglist 
    of the last column"""

    filtered_list = []

    for x in range(len(d.tags)):
        row_cont = d.tags.loc[x]
        if tag in row_cont:
            filtered_list.append(x)

    filtered = d[d.index.isin(filtered_list)]
    len_filtered = (len(filtered))
    print(len_filtered)

    return filtered


In [196]:
filt_list(df, 'Feminism')

2


,subject,summary,date_str,address,date_dt,day,tags
435,Mary Wollstonecraft,"Melvyn Bragg and guests John Mullan, Karen O'B...",31-12-2009,https://www.bbc.co.uk/programmes/b00pg5dr,2009-12-31,Thursday,"[Enlightenment, 18th Century, Philosophy, Hist..."
863,Feminism,Melvyn Bragg and guests discuss one of the mos...,07-01-1999,https://www.bbc.co.uk/programmes/p00545b0,1999-01-07,Thursday,"[20th Century, Philosophy, Culture, Feminism]"
